<a href="https://colab.research.google.com/github/Brownwang0426/RGRL/blob/main/CartPole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing requirements

In [ ]:
!sudo apt-get install python3.10

In [ ]:
!pip install pandas==2.0.3 numpy==1.25.2 scipy==1.11.4 swig==4.2.1 ufal.pybox2d==2.3.10.3 gym==0.25.2 pygame==2.5.2 tqdm torch==2.0.1

# Importing modules

In [7]:
import gym

import numpy as np
import math
from scipy.special import softmax

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, TensorDataset

import csv

import multiprocessing as mp
import os
import sys
import copy
import random
import gc
import time
from tqdm import tqdm


# Checking cuda

In [ ]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    device_index = 0
    device = torch.device(f"cuda:{device_index}")
    print('using cuda...')
else:
    device = torch.device("cpu")
    print('using cpu...')
assert device != torch.device("cpu") # Sorry, but we really recommend you to run it on GPU :-) Nvidia needs your money :-)

In [9]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

# Class for building model

In [10]:
class build_model(nn.Module):
    def __init__(self,
                 input_neuron_size_,
                 hidden_neuron_size,
                 input_neuron_size,
                 input_sequence_size,
                 output_neuron_size,
                 neural_type,
                 num_layers,
                 num_heads,
                 hidden_activation,
                 output_activation,
                 initializer,
                 optimizer,
                 loss,
                 drop_rate,
                 alpha,
                 mask_value):

        super(build_model, self).__init__()

        self.input_neuron_size_   = int(input_neuron_size_)
        self.hidden_neuron_size   = int(hidden_neuron_size)
        self.input_neuron_size    = int(input_neuron_size)
        self.input_sequence_size  = int(input_sequence_size)
        self.output_neuron_size   = int(output_neuron_size)
        self.neural_type          = neural_type
        self.num_heads            = num_heads

        self.hidden_activation    = hidden_activation
        self.output_activation    = output_activation
        self.initializer          = initializer
        self.optimizer            = optimizer
        self.loss                 = loss
        self.drop_rate            = drop_rate
        self.alpha                = alpha
        self.mask_value           = mask_value

        self.bias = False

        self.num_layers = num_layers

        neural_types = {
            'rnn': nn.RNN,
            'gru': nn.GRU,
            'lstm': nn.LSTM
        }

        self.fully_connected_layer_in_0      = nn.Linear(self.input_neuron_size_, self.hidden_neuron_size, bias=self.bias)
        self.fully_connected_layer_in_1      = nn.Linear(self.hidden_neuron_size, self.hidden_neuron_size, bias=self.bias)

        self.recurrent_layer_0               = neural_types[neural_type.lower()](self.input_neuron_size, self.hidden_neuron_size, num_layers=self.num_layers, batch_first=False, bias=self.bias, dropout=self.drop_rate)

        self.fully_connected_layer_0         = nn.Linear(self.hidden_neuron_size * self.input_sequence_size, self.output_neuron_size, bias=self.bias)

        # Activation functions
        self.hidden_activation = self.get_activation(self.hidden_activation)
        self.output_activation = self.get_activation(self.output_activation)

        # Initialize weights for fully connected layers
        self.initialize_weights(self.initializer  )

        # Optimizer
        optimizers = {
            'adam': optim.Adam,
            'sgd': optim.SGD,
            'rmsprop': optim.RMSprop
        }
        self.selected_optimizer = optimizers[self.optimizer.lower()](self.parameters(), lr=self.alpha)

        # Loss function
        losses = {
            'mean_squared_error': torch.nn.MSELoss(),
            'binary_crossentropy': torch.nn.BCELoss()
        }
        self.loss_function = losses[self.loss .lower()]

        # Loss function
        losses = {
            'mean_squared_error': torch.nn.MSELoss(reduction='none'),
            'binary_crossentropy': torch.nn.BCELoss(reduction='none')
        }
        self.loss_function_ = losses[self.loss .lower()]


    def forward(self, initial_hidden, x, padding_mask):

        h  = self.fully_connected_layer_in_0(initial_hidden)
        h  = self.hidden_activation(h)
        h  = self.fully_connected_layer_in_1(h)
        h  = self.hidden_activation(h)
        h  = torch.unsqueeze(h, dim=0).repeat(self.num_layers, 1, 1)

        out        = x.permute(1, 0, 2)
        lengths    = (out != self.mask_value).any(dim=2).sum(dim=0).cpu().long() # since x is (sequence_length, batch_size, input_size), we should use sum(dim=0)
        out        = rnn_utils.pack_padded_sequence(out, lengths, batch_first=False, enforce_sorted=False)
        # Forward propagate RNN
        if self.neural_type == 'lstm':
            out, h   = self.recurrent_layer_0(out, (h, h))
        else:
            out, h   = self.recurrent_layer_0(out, h)

        out, _     = rnn_utils.pad_packed_sequence(out, batch_first=False)
        padding    = (0, 0, 0, 0, 0, self.input_sequence_size - out.size(0))
        out        = F.pad(out, padding, "constant", 0)
        out        = out.permute(1, 0, 2)

        out    = torch.flatten(out, start_dim=1)
        out    = self.fully_connected_layer_0(out)
        out    = self.output_activation(out)

        return out 




    def get_activation(self,  activation):
        activations = {
            'relu': nn.ReLU(),
            'leaky_relu': nn.LeakyReLU(),
            'sigmoid': nn.Sigmoid(),
            'tanh': nn.Tanh()
        }
        return activations[ activation.lower()]

    def initialize_weights(self, initializer):
        initializers = {
            'random_uniform': nn.init.uniform_,
            'random_normal': nn.init.normal_,
            'glorot_uniform': nn.init.xavier_uniform_,
            'glorot_normal': nn.init.xavier_normal_,
            'xavier_uniform': nn.init.xavier_uniform_,
            'xavier_normal': nn.init.xavier_normal_
        }
        initializer = initializers[initializer.lower()]
        for layer in self.children():
            if isinstance(layer, nn.Linear):
                initializer(layer.weight)

# Function for updating pre-activated actions using error backprop

In [11]:

def update_pre_activated_actions(epoch_for_deducing,
                                 model_loader,
                                 state,
                                 pre_activated_actions,
                                 desired_reward,
                                 beta):

    model_loader_copy = copy.deepcopy(model_loader)

    for epoch in range(epoch_for_deducing):

        random.shuffle(model_loader_copy)

        for model in model_loader_copy:

            actions = torch.sigmoid(pre_activated_actions)

            model.train()
            actions = actions.clone().detach().requires_grad_(True)
            if actions.grad is not None:
                actions.grad.zero_()
            for param in model.parameters():
                param.requires_grad = False

            loss_function = model.loss_function
            output        = model(state, actions, padding_mask=None)
            total_loss    = loss_function(output, desired_reward)
            total_loss.backward() # get grad

            pre_activated_actions -= actions.grad * (1 - actions) * actions * beta # update params

    return pre_activated_actions




# Function for updating model using error backprop

Elastic weight consolidation:
https://arxiv.org/pdf/1612.00796

In [12]:
# traditional EWC
def EWC_loss(EWC_lambda, model, prev_model, prev_gradient_matrix):
    model_param      = model.state_dict()
    prev_model_param = prev_model.state_dict()
    loss = 0
    for name, param in model.named_parameters():
        diagonal_fisher_matrix = prev_gradient_matrix[name] ** 2
        param_diff             = (model_param[name] - prev_model_param[name]) ** 2
        loss                  += (diagonal_fisher_matrix * param_diff).sum()
    return EWC_lambda * loss




def update_model(epoch_for_learning,
                 model,
                 train_loader,
                 train_loader_,
                 prev_model,
                 prev_gradient_matrix,
                 EWC_lambda):

    for epoch in range(epoch_for_learning):

        for state, actions, reward, next_state, padding_mask in train_loader:

            model.train()
            selected_optimizer = model.selected_optimizer
            selected_optimizer.zero_grad()

            loss_function        = model.loss_function
            output               = model(state, actions, padding_mask)
            total_loss           = loss_function(output, reward) 
            total_loss          += EWC_loss(EWC_lambda, model, prev_model, prev_gradient_matrix)
            total_loss.backward()     # get grad

            selected_optimizer.step() # update params




    # training and updating present gradient_matrix
    gradient_matrix = {name: torch.zeros_like(param) for name, param in model.named_parameters()}

    for state, actions, reward, next_state, padding_mask in train_loader_:

        model.train()
        selected_optimizer = model.selected_optimizer
        selected_optimizer.zero_grad()

        loss_function        = model.loss_function
        output               = model(state, actions, padding_mask)
        total_loss           = loss_function(output, reward) 
        total_loss.backward()        # get grad

        for name, param in model.named_parameters():
            gradient_matrix[name] += param.grad

    gradient_matrix = {name: param / len(train_loader) for name, param in gradient_matrix.items()}




    return model, gradient_matrix

# Function for updating error for prioritized experience replay

In [13]:
def update_error(model,
                 train_loader_,
                 device):


    for state, actions, reward, next_state, padding_mask in train_loader_:

        model.train()
        selected_optimizer = model.selected_optimizer
        selected_optimizer.zero_grad()

        loss_function        = model.loss_function_
        output               = model(state, actions, padding_mask)
        total_loss           = loss_function(output, reward).detach().to(device)
        total_loss           = torch.sum(total_loss, axis=1)

    return total_loss

# Function for re-initializing action value in each step

In [14]:
def initialize_pre_activated_actions(init, noise_t, noise_r, shape):
    input = 0
    if   init == "random_uniform":
        for _ in range(noise_t):
            input += np.array([  np.random.uniform(low=0, high=1, size=shape)    ]) * noise_r
    elif init == "random_normal":
        for _ in range(noise_t):
            input += np.array([  np.random.normal(loc=0.0, scale= 1, size= shape )    ])  * noise_r
    elif init == "glorot_uniform":
        for _ in range(noise_t):
            limit = np.sqrt(6 / (shape[1] + shape[1]))
            input += np.array([  np.random.uniform(low=-limit, high=limit, size=shape)    ])  * noise_r
    elif init == "glorot_normal":
        for _ in range(noise_t):
            input += np.array([  np.random.normal(loc=0.0, scale= np.sqrt(2 / (shape[1] + shape[1])) , size= shape )    ])  * noise_r
    elif init == "xavier_uniform":
        for _ in range(noise_t):
            limit = np.sqrt(6 / (shape[1] + shape[1]))
            input += np.array([  np.random.uniform(low=-limit, high=limit, size=shape)    ])  * noise_r
    elif init == "xavier_normal":
        for _ in range(noise_t):
            input += np.array([  np.random.normal(loc=0.0, scale= np.sqrt(2 / (shape[1] + shape[1])) , size= shape )    ])  * noise_r
    return input

# Function for sequentializing state, action and reward

In [15]:
# I don't know why the following multi-processing does not work... But I kept it just for later investigation.

def process_time_size(params):
    state_list, action_list, reward_list, time_size, time = params

    sequentialized_state_list      = []
    sequentialized_action_list     = []
    sequentialized_reward_list     = []
    sequentialized_next_state_list = []

    if time_size > len(state_list[:-1]):
        time_size = len(state_list[:-1])
    else:
      pass

    time_size_ = time_size
    for i in range(len(reward_list[:])):
        sequentialized_state_list.append(       state_list [ i ] )
        sequentialized_action_list.append(      action_list[ i : i+time_size_]  )
        sequentialized_reward_list.append(      reward_list[ i + len(action_list[i:i+time_size_]) - 1 ]  )
        sequentialized_next_state_list.append(  state_list [ i + len(action_list[i:i+time_size_])     ]  )

    # a more sophisticated method
    # for j in range(time_size):
    #     time_size_ = j+1
    #     if time_size_== 1:
    #         for i in range(len(reward_list[:])):
    #             sequentialized_state_list.append(       state_list [ i ] )
    #             sequentialized_action_list.append(      action_list[ i : i+time_size_]  )
    #             sequentialized_reward_list.append(      reward_list[ i + len(action_list[i:i+time_size_]) - 1 ]  )
    #             sequentialized_next_state_list.append(  state_list [ i + len(action_list[i:i+time_size_])     ]  )
    #     else:
    #         for i in range(len(reward_list[:-time_size_+1])):
    #             sequentialized_state_list.append(       state_list [ i ] )
    #             sequentialized_action_list.append(      action_list[ i : i+time_size_]  )
    #             sequentialized_reward_list.append(      reward_list[ i + len(action_list[i:i+time_size_]) - 1 ]  )
    #             sequentialized_next_state_list.append(  state_list [ i + len(action_list[i:i+time_size_])     ]  )

    return (sequentialized_state_list, sequentialized_action_list, sequentialized_reward_list, sequentialized_next_state_list)




def sequentialize(state_list, action_list, reward_list, time_size):
    # Prepare the parameters for each process
    params_list = [(state_list, action_list, reward_list, time_size, time) for time in range(time_size)]

    # Use multiprocessing Pool to process chunks in parallel
    with mp.Pool(processes=mp.cpu_count()) as pool:
        results = pool.map(process_time_size, params_list)

    # Aggregate results
    sequentialized_state_list = []
    sequentialized_action_list = []
    sequentialized_reward_list = []
    sequentialized_next_state_list = []

    for result in results:
        s_states, s_actions, s_rewards, s_next_states = result
        sequentialized_state_list.extend(s_states)
        sequentialized_action_list.extend(s_actions)
        sequentialized_reward_list.extend(s_rewards)
        sequentialized_next_state_list.extend(s_next_states)

    return sequentialized_state_list, sequentialized_action_list, sequentialized_reward_list, sequentialized_next_state_list


In [16]:
def sequentialize(state_list, action_list, reward_list, time_size):

    sequentialized_state_list       = []
    sequentialized_action_list      = []
    sequentialized_reward_list      = []
    sequentialized_next_state_list  = []

    if time_size > len(state_list[:-1]):
        time_size = len(state_list[:-1])
    else:
      pass

    time_size_ = time_size
    for i in range(len(reward_list[:])):
        sequentialized_state_list.append(       state_list [ i ] )
        sequentialized_action_list.append(      action_list[ i : i+time_size_]  )
        sequentialized_reward_list.append(      reward_list[ i + len(action_list[i:i+time_size_]) - 1 ]  )
        sequentialized_next_state_list.append(  state_list [ i + len(action_list[i:i+time_size_])     ]  )

    # a more sophisticated method
    # for j in range(time_size):
    #     time_size_ = j+1
    #     if time_size_== 1:
    #         for i in range(len(reward_list[:])):
    #             sequentialized_state_list.append(       state_list [ i ] )
    #             sequentialized_action_list.append(      action_list[ i : i+time_size_]  )
    #             sequentialized_reward_list.append(      reward_list[ i + len(action_list[i:i+time_size_]) - 1 ]  )
    #             sequentialized_next_state_list.append(  state_list [ i + len(action_list[i:i+time_size_])     ]  )
    #     else:
    #         for i in range(len(reward_list[:-time_size_+1])):
    #             sequentialized_state_list.append(       state_list [ i ] )
    #             sequentialized_action_list.append(      action_list[ i : i+time_size_]  )
    #             sequentialized_reward_list.append(      reward_list[ i + len(action_list[i:i+time_size_]) - 1 ]  )
    #             sequentialized_next_state_list.append(  state_list [ i + len(action_list[i:i+time_size_])     ]  )

    return sequentialized_state_list, sequentialized_action_list, sequentialized_reward_list, sequentialized_next_state_list

In [17]:
def save_performance_to_csv(performance_log, filename='performance_log.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Episode', 'Summed_Reward'])
        writer.writerows(performance_log)

# Function for vectorizing
Crucial function regarding how you manipulate or shape your state, action and reward

- It's essential to choose between immediate rewards and summed rewards for training your agent. If the current state doesn't encapsulate all crucial past information, using immediate rewards is advisable. This approach prevents confusion caused by varying summed rewards for the same state.

- As for reward shaping, it is recommended to increase your reward upper and decrease your reward lower bound.

In [18]:

def quantifying(array_size, init, interval, input):
    array = np.zeros(array_size)
    index = int( (input - init) // interval + 1)
    if index >= 0:
        array[ : index] = 1
    return array

def vectorizing_state(state):      # Reminder: change this for your specific task ⚠️⚠️⚠️
    state_0 = quantifying(100, -2.5, 0.050, state[0])
    state_1 = quantifying(100, -3.75, 0.075, state[1])
    state_2 = quantifying(100, -0.375, 0.0075, state[2])
    state_3 = quantifying(100, -3.75, 0.075, state[3])
    state_4 = quantifying(100, 0, 10, 0)
    state   = np.atleast_2d(np.concatenate((state_0, state_1, state_2, state_3, state_4)))
    return state

def vectorizing_action(action_size, action_argmax):  # Reminder: change this for your specific task ⚠️⚠️⚠️
    return np.eye(action_size)[action_argmax]

def vectorizing_reward(state, reward, summed_reward, done, reward_size):       # Reminder: change this for your specific task ⚠️⚠️⚠️
    if done:
        reward = np.zeros(reward_size)
    else:
        reward = np.ones(reward_size)
    return reward


# Function for data preparation

In [19]:
def obtain_tensor_from_list(sequentialized_state_list,
                            sequentialized_actions_list,
                            sequentialized_reward_list,
                            sequentialized_next_state_list,
                            device,
                            time_size,
                            mask_value,
                            num_heads):

    # Convert lists to tensors directly on the desired device and data type
    state_tensor = torch.tensor(np.array(sequentialized_state_list), dtype=torch.float).to(device)
    reward_tensor = torch.tensor(np.array(sequentialized_reward_list), dtype=torch.float).to(device)
    next_state_tensor = torch.tensor(np.array(sequentialized_next_state_list), dtype=torch.float).to(device)

    # Pad and stack actions_tensor efficiently
    actions_list = []
    for arr in sequentialized_actions_list:
        tensor_arr = torch.tensor(np.array(arr), dtype=torch.float).to(device)
        # Pad tensor only once per tensor
        if tensor_arr.size(0) < time_size:
            padded_arr = F.pad(tensor_arr,
                               (0, 0, 0, time_size - tensor_arr.size(0)),
                               mode='constant',
                               value=mask_value)
        else:
            padded_arr = tensor_arr
        actions_list.append(padded_arr)
    actions_tensor = torch.stack(actions_list).to(device)

    # Compute row_mask and padding_mask efficiently
    row_mask = (actions_tensor == mask_value).all(dim=-1)
    padding_mask = row_mask.to(dtype=torch.bool)
    padding_mask = padding_mask.to(device)

    return state_tensor, actions_tensor, reward_tensor, next_state_tensor, padding_mask

In [20]:
def obtain_tensor_according_to_TD_error(state_tensor, actions_tensor, reward_tensor, next_state_tensor, padding_mask, index_list):

    # Use advanced indexing to select elements based on random_index
    state_tensor = state_tensor[index_list]
    actions_tensor = actions_tensor[index_list]
    reward_tensor = reward_tensor[index_list]
    next_state_tensor = next_state_tensor[index_list]
    padding_mask = padding_mask[index_list]

    return state_tensor, actions_tensor, reward_tensor, next_state_tensor, padding_mask

# Control board

Crucial variables regarding how your agent will learn in the environment

- In some environments, it is crucial to increase your "max_steps_for_each_episode" so that your agent can "live long enough" to obatin some better rewards to gradually and heuristically learn better strategy.



In [21]:
game_name = 'CartPole-v1'                # Reminder: change this for your specific task ⚠️⚠️⚠️
max_steps_for_each_episode = 2000        # Reminder: change this for your specific task ⚠️⚠️⚠️


ensemble_size = 10                       # Reminder: change this value to see the impact of MWM-SGD ◀️◀️◀️
state_size =  500                        # Reminder: change this for your specific task ⚠️⚠️⚠️
hidden_size = 100                        # Reminder: change this for your specific task ⚠️⚠️⚠️
action_size = 2                          # Reminder: change this for your specific task ⚠️⚠️⚠️
time_size = 10                           # Reminder: change this for your specific task ⚠️⚠️⚠️
reward_size = 100                        # Reminder: change this for your specific task ⚠️⚠️⚠️
neural_type = 'gru'                      # rnn gru lstm; att
num_layers = 2                           # 1, 2, 3, 4, etc.
num_heads = None                         # None for non-attention; should be able to divide hidden_size for attention
hidden_activation = 'tanh'               # relu leaky_relu sigmoid tanh
output_activation = 'sigmoid'            # relu leaky_relu sigmoid tanh

init = "random_normal"                   # random_normal random_uniform xavier_normal xavier_uniform  glorot_normal  glorot_uniform
opti = 'sgd'                             # adam sgd rmsprop
loss = 'mean_squared_error'              # mean_squared_error  binary_crossentropy
drop_rate = 0.001                        # Reminder: change this value to see the impact of drop-out ◀️◀️◀️
alpha = 0.1                              # learning rate
epoch_for_learning = 50                  # Reminder: change this for your specific task ⚠️⚠️⚠️
mask_value = sys.maxsize                 # mask value
batch_size = 1
load_pre_model = False


noise_t = 1               # gaussian noise
noise_r = 0.1             # smaller value encourages agent to exploit experience while larger value encourages agent to explore
beta = 0.1
epoch_for_deducing =  int(500/ensemble_size)    # Reminder: change this for your specific task ⚠️⚠️⚠️


episode_for_training             = 100000
interval_for_initiating_learning = 1     # batch size for batch offline learning, Reminder: change this for your specific task ⚠️⚠️⚠️
replay_t = 1000                          # prioritized experience replay samples, Reminder: change this for your specific task ⚠️⚠️⚠️
EWC_lambda = 1                           # elastic weight control lambda


episode_for_testing = 100
render_for_human = True


suffix                      = f"game={game_name}_type={neural_type}_ensemble={ensemble_size:05d}_drop={drop_rate:.5f}_learn={epoch_for_learning:05d}_interval={interval_for_initiating_learning:05d}_deduce={epoch_for_deducing:05d}_lambda={EWC_lambda:05d}"
directory                   = f'/content/result/{game_name}/'
model_directory             = f'/content/result/{game_name}/model_{suffix}'+'_%s.h5'
performance_log_directory   = f'/content/result/{game_name}/performace_log_{suffix}.csv'

# Deducing > Learning


Creating or loading models

In [22]:

if not os.path.exists(directory):
    os.makedirs(directory)

if load_pre_model == False:

    model_loader = []
    for _ in range(ensemble_size):
        model = build_model(state_size,
                            hidden_size,
                            action_size,
                            time_size,
                            reward_size,
                            neural_type,
                            num_layers,
                            num_heads,
                            hidden_activation,
                            output_activation,
                            init,
                            opti,
                            loss,
                            drop_rate,
                            alpha,
                            mask_value)
        model.to(device)
        model_loader.append(model)

elif load_pre_model == True:

    model_loader = []
    for _ in range(ensemble_size):
        model = build_model(state_size,
                            hidden_size,
                            action_size,
                            time_size,
                            reward_size,
                            neural_type,
                            num_layers,
                            num_heads,
                            hidden_activation,
                            output_activation,
                            init,
                            opti,
                            loss,
                            drop_rate,
                            alpha,
                            mask_value)
        model.to(device)
        model_loader.append(model)

    for i in range(len(model_loader)):
        model_loader[i].load_state_dict(torch.load( model_directory  % i ))


Creating Streams

In [23]:
stream_list = []
for _ in range(ensemble_size):
    stream  = torch.cuda.Stream()
    stream_list.append(stream)


Creating intial gradient matrices

In [24]:

prev_model_loader = copy.deepcopy(model_loader)

prev_gradient_matrix_loader = []
for model in model_loader:
    gradient_matrix = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
    prev_gradient_matrix_loader.append( gradient_matrix )


Creating desired reward

In [25]:
desired_reward = np.atleast_2d(np.ones(reward_size))
desired_reward = torch.tensor(desired_reward, dtype=torch.float).to(device)

Putting all the previous works into play

In [26]:

performance_log = []
performance_log.append([0, 0])

for training_episode in tqdm(range(episode_for_training)):

    # initializing short term experience replay buffer
    short_term_state_list  = []
    short_term_action_list = []
    short_term_reward_list = []

    # initializing environment
    env           = gym.make(game_name)
    env._max_episode_steps = max_steps_for_each_episode
    state         = env.reset()
    summed_reward = 0

    # observing state
    state = vectorizing_state(state)
    short_term_state_list.append(state[0])

    done = False
    while not done:

        # initializing and updating actions
        state                 = torch.tensor(state, dtype=torch.float).to(device)
        pre_activated_actions = initialize_pre_activated_actions(init, noise_t, noise_r, (time_size, action_size))
        pre_activated_actions = torch.tensor(pre_activated_actions, dtype=torch.float).to(device)
        pre_activated_actions = update_pre_activated_actions(epoch_for_deducing,
                                                             model_loader,
                                                             state,
                                                             pre_activated_actions,
                                                             desired_reward,
                                                             beta)
        action_argmax    = int(torch.argmax(pre_activated_actions[0, 0]))
        action           = vectorizing_action(action_size, action_argmax)
        short_term_action_list.append(action)

        # executing action
        state, reward, done, info = env.step(action_argmax)

        # observing actual reward
        summed_reward += reward
        reward = vectorizing_reward(state, reward, summed_reward, done, reward_size)
        short_term_reward_list.append(reward)

        # observing state
        state = vectorizing_state(state)
        short_term_state_list.append(state[0])

        if done:
            print(f'Episode {training_episode+1}: Summed_Reward = {summed_reward}')
            performance_log.append([training_episode+1, summed_reward])
            save_performance_to_csv(performance_log, performance_log_directory)
            break




    env.close()




    # sequentialize short term experience replay buffer and then save it to long term experience replay buffer
    short_term_sequentialized_state_list, \
    short_term_sequentialized_actions_list, \
    short_term_sequentialized_reward_list, \
    short_term_sequentialized_next_state_list = sequentialize(short_term_state_list, short_term_action_list, short_term_reward_list, time_size )
    
    short_term_sequentialized_state_tensor,\
    short_term_sequentialized_actions_tensor,\
    short_term_sequentialized_reward_tensor,\
    short_term_sequentialized_next_state_tensor,\
    short_term_sequentialized_padding_mask = obtain_tensor_from_list(short_term_sequentialized_state_list,
                                                                     short_term_sequentialized_actions_list,
                                                                     short_term_sequentialized_reward_list,
                                                                     short_term_sequentialized_next_state_list,
                                                                     device,
                                                                     time_size,
                                                                     mask_value,
                                                                     num_heads)
    if training_episode==0:
        long_term_sequentialized_state_tensor      = copy.deepcopy(short_term_sequentialized_state_tensor)
        long_term_sequentialized_actions_tensor    = copy.deepcopy(short_term_sequentialized_actions_tensor)
        long_term_sequentialized_reward_tensor     = copy.deepcopy(short_term_sequentialized_reward_tensor)
        long_term_sequentialized_next_state_tensor = copy.deepcopy(short_term_sequentialized_next_state_tensor)
        long_term_sequentialized_padding_mask      = copy.deepcopy(short_term_sequentialized_padding_mask)
    else:
        long_term_sequentialized_state_tensor      = torch.cat((long_term_sequentialized_state_tensor     , short_term_sequentialized_state_tensor     ), dim=0)
        long_term_sequentialized_actions_tensor    = torch.cat((long_term_sequentialized_actions_tensor   , short_term_sequentialized_actions_tensor   ), dim=0)
        long_term_sequentialized_reward_tensor     = torch.cat((long_term_sequentialized_reward_tensor    , short_term_sequentialized_reward_tensor    ), dim=0)
        long_term_sequentialized_next_state_tensor = torch.cat((long_term_sequentialized_next_state_tensor, short_term_sequentialized_next_state_tensor), dim=0)
        long_term_sequentialized_padding_mask      = torch.cat((long_term_sequentialized_padding_mask     , short_term_sequentialized_padding_mask     ), dim=0)
        



    # batch offline learning
    if (training_episode+1) % interval_for_initiating_learning == 0:




        # creating samples for prioritized experience replay buffer:
        dataset      = TensorDataset(long_term_sequentialized_state_tensor     ,
                                     long_term_sequentialized_actions_tensor   ,
                                     long_term_sequentialized_reward_tensor    ,
                                     long_term_sequentialized_next_state_tensor,
                                     long_term_sequentialized_padding_mask     )
        data_loader_ = DataLoader(dataset, batch_size = len(dataset), shuffle=False)
        total_temporal_difference_error = 0
        for i, model in enumerate(model_loader):
            with torch.cuda.stream(stream_list[i]):
                temporal_difference_error        = update_error(model, 
                                                                data_loader_,
                                                                device)
                total_temporal_difference_error += temporal_difference_error
        torch.cuda.synchronize()
        index_list         = torch.argsort(total_temporal_difference_error)[-replay_t:]
        selected_sequentialized_state_tensor,\
        selected_sequentialized_actions_tensor,\
        selected_sequentialized_reward_tensor,\
        selected_sequentialized_next_state_tensor,\
        selected_sequentialized_padding_mask = obtain_tensor_according_to_TD_error(long_term_sequentialized_state_tensor     ,
                                                                                   long_term_sequentialized_actions_tensor   ,
                                                                                   long_term_sequentialized_reward_tensor    ,
                                                                                   long_term_sequentialized_next_state_tensor,
                                                                                   long_term_sequentialized_padding_mask     ,
                                                                                   index_list)
        



        # starting learning with elastic weight control
        dataset      = TensorDataset(selected_sequentialized_state_tensor,
                                     selected_sequentialized_actions_tensor,
                                     selected_sequentialized_reward_tensor,
                                     selected_sequentialized_next_state_tensor,
                                     selected_sequentialized_padding_mask)
        data_loader  = DataLoader(dataset, batch_size = batch_size, shuffle=True)
        data_loader_ = DataLoader(dataset, batch_size = len(dataset), shuffle=False)
        gradient_matrix_loader = [''] * len(model_loader)
        for i, model in enumerate(model_loader):
            with torch.cuda.stream(stream_list[i]):
                model, gradient_matrix    = update_model(epoch_for_learning,
                                                         model,
                                                         data_loader,
                                                         data_loader_,
                                                         prev_model_loader[i],
                                                         prev_gradient_matrix_loader[i],
                                                         EWC_lambda)
                model_loader[i]           = model
                gradient_matrix_loader[i] = gradient_matrix
        torch.cuda.synchronize()
        prev_model_loader           = copy.deepcopy(model_loader)
        prev_gradient_matrix_loader = copy.deepcopy(gradient_matrix_loader)




        # saving:
        for i in range(len(model_loader)):
            torch.save(model_loader[i].state_dict(), model_directory % i)


        gc.collect()
        torch.cuda.empty_cache()



# Deducing (testing)

Loading models

In [ ]:
model_loader = []
for _ in range(ensemble_size):
    model = build_model(state_size,
                        hidden_size,
                        action_size,
                        time_size,
                        reward_size,
                        neural_type,
                        num_layers,
                        num_heads,
                        hidden_activation,
                        output_activation,
                        init,
                        opti,
                        loss,
                        drop_rate,
                        alpha,
                        mask_value)
    model.to(device)
    model_loader.append(model)

for i in range(len(model_loader)):
    model_loader[i].load_state_dict(torch.load(model_directory % i))

Creating desired reward ... again

In [ ]:
desired_reward = np.atleast_2d(np.ones(reward_size))
desired_reward = torch.tensor(desired_reward, dtype=torch.float).to(device)

Putting all the previous works into play ... again

But this time the agent does not learn

In [ ]:
total_summed_reward = 0

for testing_episode in range(episode_for_testing):

    if render_for_human == True:
        env = gym.make( game_name, render_mode="human")
    else:
        env = gym.make( game_name)
    env._max_episode_steps = max_steps_for_each_episode
    state                  = env.reset()
    if render_for_human == True:
        env.render()
    summed_reward = 0

    state = vectorizing_state(state)

    done = False
    while not done:

        state                 = torch.tensor(state, dtype=torch.float).to(device)
        pre_activated_actions = initialize_pre_activated_actions(init, noise_t, noise_r, (time_size, action_size))
        pre_activated_actions = torch.tensor(pre_activated_actions, dtype=torch.float).to(device)
        pre_activated_actions = update_pre_activated_actions(epoch_for_deducing,
                                                             model_loader,
                                                             state,
                                                             pre_activated_actions,
                                                             desired_reward,
                                                             beta)
        action_argmax    = int(torch.argmax(pre_activated_actions[0, 0]))

        state, reward, done,  info = env.step(action_argmax)
        if render_for_human == True:
            env.render()

        summed_reward += reward

        state = vectorizing_state(state)

        if done:
            break


    env.close()

    print("Summed reward:", summed_reward)
    print(f'Episode: {testing_episode + 1}')
    print('Everaged summed reward:')
    total_summed_reward += summed_reward
    print(total_summed_reward/(testing_episode + 1))

